In [1]:
# Carga manual del modelo y configuración del esquema (sin pipeline)

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
import numpy as np

# Nombre del modelo

model_id = "PipableAI/pip-sql-1.3b"

# Tokenizador y modelo
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Uso de CPU (compatible con EC2 t3.medium o Mac M2)
device = torch.device("cpu")
model.to(device)

# Prompt del esquema RAWG
schema_prompt = """
### Base de datos PostgreSQL: RAWG Video Games

Tablas principales:
- games(id_game, name, released, rating, rating_top, ratings_count, metacritic, tba, playtime, updated, esrb_rating_id)
- genres(id_genre, name)
- game_genres(id_game, id_genre)
- platforms(id_platform, name)
- game_platforms(id_game, id_platform, released_at)
- parent_platforms(id_parent_platform, name)
- game_parent_platforms(id_game, id_parent_platform)
- tags(id_tag, name)
- game_tags(id_game, id_tag)
- stores(id_store, name)
- game_stores(id_game, id_store)
- ratings(id_rating, title, count, percent)
- game_ratings(id_game, id_rating)
- esrb_ratings(id_esrb_rating, name)
- game_added_by_status(id_game, status, count)

Relaciones clave:
- Un juego puede tener múltiples géneros, plataformas, tags, tiendas y ratings.
- La tabla `game_platforms` enlaza juegos con plataformas y fechas de lanzamiento específicas.
- La tabla `ratings` define tipos de calificación como “exceptional”, “meh”, etc.
- `game_added_by_status` representa cuántos usuarios tienen un juego en estados como “playing” o “completed”.

Restricciones semánticas:
- Las preguntas deben referirse a datos relacionados con videojuegos, plataformas, géneros, puntuaciones, fechas de lanzamiento, etiquetas u otras métricas del ecosistema RAWG.
- Si la pregunta no está relacionada con videojuegos o con esta base de datos, no debe generarse ninguna consulta SQL.
"""

2025-08-04 10:32:16.739155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754296336.760999   15175 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754296336.767312   15175 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754296336.782947   15175 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754296336.782968   15175 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754296336.782971   15175 computation_placer.cc:177] computation placer alr

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:  14%|#4        | 828M/5.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
# Función generadora
def question_to_sql(user_question):
    prompt = f"{schema_prompt}\n\n### Pregunta:\n{user_question}\n\n### SQL:"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=256)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Buscar el inicio real del bloque SQL
    match = re.search(r"### SQL:\s*(select.+?)(?:\n###|\Z)", decoded, re.IGNORECASE | re.DOTALL)
    if match:
        sql_code = match.group(1).strip()
        return sql_code
    else:
        return "[ERROR] No se pudo extraer una consulta SQL válida."

In [14]:
def validar_sql_generada(sql_code):
    """
    Verifica si la consulta SQL generada es válida para su ejecución.

    Criterios:
    - Debe comenzar con SELECT
    - No debe contener operaciones peligrosas (DROP, DELETE, UPDATE, INSERT, etc.)
    - Debe contener al menos una tabla válida del esquema
    - No debe estar vacía
    """
    if not sql_code or not isinstance(sql_code, str):
        return False, "La consulta SQL está vacía o no es una cadena de texto."

    sql_lower = sql_code.strip().lower()

    sql_trimmed = re.sub(r"^\s+|(--.*\n)+", "", sql_lower, flags=re.MULTILINE)
    if not sql_trimmed.startswith("select"):
        return False, "Solo se permiten consultas SELECT."


    # Evitar consultas peligrosas
    if re.search(r"\b(drop|delete|update|insert|alter|truncate)\b", sql_lower):
        return False, "La consulta contiene operaciones peligrosas no permitidas."

    # Comprobar si hace referencia al menos a una tabla esperada
    tablas_validas = [
        "games", "genres", "game_genres", "platforms", "game_platforms", "tags",
        "game_tags", "stores", "game_stores", "ratings", "game_ratings", 
        "esrb_ratings", "game_added_by_status", "parent_platforms", "game_parent_platforms"
    ]

    if not any(tabla in sql_lower for tabla in tablas_validas):
        return False, "La consulta no hace referencia a ninguna tabla conocida."

    return True, "Consulta válida"

In [15]:
# ✍️ Prueba aquí tus preguntas
question = "what are the best 10 rated games?"
generated_sql = question_to_sql(question)
print("Pregunta:", question)
print("\nSQL generada:\n", generated_sql)
print(validar_sql_generada(generated_sql))
# Validación de la consulta SQL generada
# is_valid, validation_message = validar_sql_generada(generated_sql)
# print("\nValidación de la consulta SQL:")
# if is_valid:
#     print("✅ La consulta es válida.")
# else:
#     print("❌ La consulta no es válida:", validation_message)

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


KeyboardInterrupt: 